# 6. Test Train Data Preparation #
## For Brownlow Predictor Project ##

Does 5Fold test train split for each of the 6 different types of data.
Four different type of preparation undergone:
- [x] Regular
- [x] Bootstrap
- [x] Two-Step
- [x] Bootstrap and Two-Step

and outputs them as individual .csv's.

*A form of trading Time for Space: By preemptively preparing the 16GB of train-test (and manipulated otherwise) data, saves a lot of computational time when scripts are run (i.e. don't have to do test-train splits or sampling.*

*Also: for the purpose of experiments maintains a controlled variable (all different models are essentially using the same rows of data for each train-split because the same seed is used*

***KFold Train-Test split is used to collect better statistics for each model (i.e. each model trained five times so unlikely for a bad model to have good statistics due to luck. Split size of 5 used to ensure 1. not too many splits (computational time: adds a factor k to the permutations later on in scripts); 2. 20% data for testing (ensures a sizeable test size); 3. 80% data for training (because 3 votes 2 votes and 1 votes make up only roughly 1/15 of the rows of data, any less then 80% of training data could produce flawed results)***

**Author: `Lang (Ron) Chen` 2021.12-2022.1**

---

*Note: As Percentages followed by Standardisation was proven to be exactly the same as Standardisation, all relevent code have bben commented out* 

**0. Import Libraries**

In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

**1. Preparing Train-Test Split data**

In [2]:
# choice = ['NormalisedData', 'StandardisedData', 'RankStandardisedData', 'PercentageData', 'PercentageStandardisedData', 'PercentageNormalisedData']
choice = ['NormalisedData', 'StandardisedData', 'RankStandardisedData', 'PercentageData', 'PercentageNormalisedData']

Train Test split done on a per-game basis because:
1. Maintains a fixed proportion of test and train cases as 3 votes, 2 votes, 1 vote
2. The model ultimately must be tested on a games based basis

1. Regular

In [3]:
%%time

# NAME = {'NormalisedData': 'N', 'StandardisedData': 'S', 'RankStandardisedData': 'RS', 'PercentageData': 'P', 
#         'PercentageStandardisedData': 'PS', 'PercentageNormalisedData': 'PN'}
NAME = {'NormalisedData': 'N', 'StandardisedData': 'S', 'RankStandardisedData': 'RS', 'PercentageData': 'P', 'PercentageNormalisedData': 'PN'}

kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

# Each different data manipulation format needs to undergo the same train-test split
for c in choice:
    filelist = os.listdir(f'./Data/{c}')[1:] # Remove the first file (an ipynb checkpoint file)
    test_train_games = [file for file in filelist if '2021' not in file] # list of games

    index = 1
    for train_games_i, test_games_i in kf.split(test_train_games): # test train split in terms of games
        
        train_games = [test_train_games[i] for i in train_games_i]
        
        test_games = [test_train_games[i] for i in test_games_i]
        
        # opens each file and concatenates them
        train_data = pd.read_csv(f'./Data/{c}/{train_games[0]}')
        
        for file in train_games[1:]:
            df = pd.read_csv(f'./Data/{c}/{file}')
            train_data = pd.concat([train_data, df], axis=0)

        
        # Uses f-strings and loop variables to automatically export data as .csv's with diffent names
        train_data.to_csv(f'Train_Data_{index} ({NAME[c]}).csv', index = False)
        
        
        # Export file list of test games (literally a Series) into a csv for ease of use in scripts
        test_game_list = pd.DataFrame({'Test Games': test_games})
        
        test_game_list.to_csv(f'Test_Games_List_{index} ({NAME[c]}).csv', index = False)

        index += 1

2. Bootstrapped (B)

In [7]:
%%time

# NAME = {'NormalisedData': 'N', 'StandardisedData': 'S', 'RankStandardisedData': 'RS', 'PercentageData': 'P', 
#         'PercentageStandardisedData': 'PS', 'PercentageNormalisedData': 'PN'}
NAME = {'NormalisedData': 'N', 'StandardisedData': 'S', 'RankStandardisedData': 'RS', 'PercentageData': 'P', 'PercentageNormalisedData': 'PN'}

kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

# Each different data manipulation format needs to undergo the same train-test split
for c in choice:
    filelist = os.listdir(f'./Data/{c}')[1:] # Remove the first file (an ipynb checkpoint file)
    test_train_games = [file for file in filelist if '2021' not in file] # list of games

    index = 1
    for train_games_i, test_games_i in kf.split(test_train_games): # test train split in terms of games
        
        train_games = [test_train_games[i] for i in train_games_i]
        
        test_games = [test_train_games[i] for i in test_games_i]
        
        # opens each file and concatenates them
        train_data = pd.read_csv(f'./Data/{c}/{train_games[0]}')
        
        for file in train_games[1:]:
            df = pd.read_csv(f'./Data/{c}/{file}')
            train_data = pd.concat([train_data, df], axis=0)
        
        # Bootstraps data
        
        # Picks out data labelled 1 vote, 2 votes, 3 votes 
        zero = train_data[train_data['Brownlow Votes'] == 0]
        one = train_data[train_data['Brownlow Votes'] == 1]
        two = train_data[train_data['Brownlow Votes'] == 2]
        three = train_data[train_data['Brownlow Votes'] == 3]
        
        # Sample them so each has same number as 0 votes
        new_one = one.sample(n = len(zero), replace = True, random_state = 42)
        new_two = two.sample(n = len(zero), replace = True, random_state = 42)
        new_three = three.sample(n = len(zero), replace = True, random_state = 42)
        
        # Add the sampled dataframes back onto zero
        bootstrapped_data = pd.concat([zero, new_one, new_two, new_three], axis = 0)
        
        
        # Uses f-strings and loop variables to automatically export data as .csv's with diffent names
        bootstrapped_data.to_csv(f'Train_Data_{index} ({NAME[c]}) (B).csv', index = False)
        
        
        # Export file list of test games (literally a Series) into a csv for ease of use in scripts
        test_game_list = pd.DataFrame({'Test Games': test_games})
        
        test_game_list.to_csv(f'Test_Games_List_{index} ({NAME[c]}) (B).csv', index = False)

        index += 1

3. Two rounds (2)

*Three ways to treat the >0 votes data in 1st step: assign them vote of 1 (min), vote of 2 (avg), vote of 3 (max).*

In [5]:
%%time

# NAME = {'NormalisedData': 'N', 'StandardisedData': 'S', 'RankStandardisedData': 'RS', 'PercentageData': 'P', 
#         'PercentageStandardisedData': 'PS', 'PercentageNormalisedData': 'PN'}
NAME = {'NormalisedData': 'N', 'StandardisedData': 'S', 'RankStandardisedData': 'RS', 'PercentageData': 'P', 'PercentageNormalisedData': 'PN'}

kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

# Each different data manipulation format needs to undergo the same train-test split
for c in choice:
    filelist = os.listdir(f'./Data/{c}')[1:] # Remove the first file (an ipynb checkpoint file)
    test_train_games = [file for file in filelist if '2021' not in file] # list of games

    index = 1
    for train_games_i, test_games_i in kf.split(test_train_games): # test train split in terms of games
        
        train_games = [test_train_games[i] for i in train_games_i]
        
        test_games = [test_train_games[i] for i in test_games_i]
        
        # opens each file and concatenates them
        train_data = pd.read_csv(f'./Data/{c}/{train_games[0]}')

        for file in train_games[1:]:
            df = pd.read_csv(f'./Data/{c}/{file}')
            train_data = pd.concat([train_data, df], axis=0)
        
        # Splits data into zero votes and more than 0 votes
        zero = train_data[train_data['Brownlow Votes'] == 0]
        
        # three copies because will relabel them with three different options
        tagged1 = train_data[train_data['Brownlow Votes'] > 0]
        tagged2 = train_data[train_data['Brownlow Votes'] > 0]
        tagged3 = train_data[train_data['Brownlow Votes'] > 0]

        tagged1['Brownlow Votes'] = tagged1['Brownlow Votes'].replace([1, 2, 3], 1)
        tagged2['Brownlow Votes'] = tagged2['Brownlow Votes'].replace([1, 2, 3], 2)
        tagged3['Brownlow Votes'] = tagged3['Brownlow Votes'].replace([1, 2, 3], 3)
        
        # concatenate them together
        first_lr_data1 = pd.concat([zero, tagged1], axis = 0)
        first_lr_data2 = pd.concat([zero, tagged2], axis = 0)
        first_lr_data3 = pd.concat([zero, tagged3], axis = 0)

        second_lr_data = train_data[train_data['Brownlow Votes'] > 0]
        
        # Export the step-1 data (3 different labels) into 3 different .csv's and also export step-2 data
        first_lr_data1.to_csv(f'Train_Data_{index} ({NAME[c]}) (2_1_1).csv', index = False)
        first_lr_data2.to_csv(f'Train_Data_{index} ({NAME[c]}) (2_1_2).csv', index = False)
        first_lr_data3.to_csv(f'Train_Data_{index} ({NAME[c]}) (2_1_3).csv', index = False)
        second_lr_data.to_csv(f'Train_Data_{index} ({NAME[c]}) (2_2).csv', index = False)
        
        
        # Export file list of test games (literally a Series) into a csv for ease of use in scripts
        test_game_list = pd.DataFrame({'Test Games': test_games})
        
        test_game_list.to_csv(f'Test_Games_List_{index} ({NAME[c]}) (2).csv', index = False)

        index += 1

C:\Users\GZCHE\anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\GZCHE\anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\GZCHE\anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

Wall time: 30min 29s


Two Rounds + Bootstrapped (2)(B)

In [7]:
%%time

# NAME = {'NormalisedData': 'N', 'StandardisedData': 'S', 'RankStandardisedData': 'RS', 'PercentageData': 'P', 
#         'PercentageStandardisedData': 'PS', 'PercentageNormalisedData': 'PN'}
NAME = {'NormalisedData': 'N', 'StandardisedData': 'S', 'RankStandardisedData': 'RS', 'PercentageData': 'P', 'PercentageNormalisedData': 'PN'}

kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

# Each different data manipulation format needs to undergo the same train-test split
for c in choice:
    filelist = os.listdir(f'./Data/{c}')[1:] # Remove the first file (an ipynb checkpoint file)
    test_train_games = [file for file in filelist if '2021' not in file] # list of games

    index = 1
    for train_games_i, test_games_i in kf.split(test_train_games): # test train split in terms of games
        
        train_games = [test_train_games[i] for i in train_games_i]
        
        test_games = [test_train_games[i] for i in test_games_i]
        
        train_data = pd.read_csv(f'./Data/{c}/{train_games[0]}')

        for file in train_games[1:]:
            df = pd.read_csv(f'./Data/{c}/{file}')
            train_data = pd.concat([train_data, df], axis=0)
        
        # Splits data into zero votes and more than 0 votes
        zero = train_data[train_data['Brownlow Votes'] == 0]
        
        # three copies because will relabel them with three different options
        tagged1 = train_data[train_data['Brownlow Votes'] > 0]
        tagged2 = train_data[train_data['Brownlow Votes'] > 0]
        tagged3 = train_data[train_data['Brownlow Votes'] > 0]

        tagged1['Brownlow Votes'] = tagged1['Brownlow Votes'].replace([1, 2, 3], 1)
        tagged2['Brownlow Votes'] = tagged2['Brownlow Votes'].replace([1, 2, 3], 2)
        tagged3['Brownlow Votes'] = tagged3['Brownlow Votes'].replace([1, 2, 3], 3)
        
        # Bootstraps data
        
        # Sample them (using the same random_state (seed) outcome is the same)
        new_tagged1 = tagged1.sample(n = len(zero), replace = True, random_state = 42)
        new_tagged2 = tagged2.sample(n = len(zero), replace = True, random_state = 42)
        new_tagged3 = tagged3.sample(n = len(zero), replace = True, random_state = 42)
        
        # concatenate them together
        first_lr_data1 = pd.concat([zero, new_tagged1], axis = 0)
        first_lr_data2 = pd.concat([zero, new_tagged2], axis = 0)
        first_lr_data3 = pd.concat([zero, new_tagged3], axis = 0)

        second_lr_data = train_data[train_data['Brownlow Votes'] > 0]
        
        # Export the step-1 data (3 different labels) into 3 different .csv's and also export step-2 data
        first_lr_data1.to_csv(f'Train_Data_{index} ({NAME[c]}) (2_1_1)(B).csv', index = False)
        first_lr_data2.to_csv(f'Train_Data_{index} ({NAME[c]}) (2_1_2)(B).csv', index = False)
        first_lr_data3.to_csv(f'Train_Data_{index} ({NAME[c]}) (2_1_3)(B).csv', index = False)
        second_lr_data.to_csv(f'Train_Data_{index} ({NAME[c]}) (2_2)(B).csv', index = False)
        
        
        # Export file list of test games (literally a Series) into a csv for ease of use in scripts
        test_game_list = pd.DataFrame({'Test Games': test_games})
        
        test_game_list.to_csv(f'Test_Games_List_{index} ({NAME[c]}) (2)(B).csv', index = False)

        index += 1

C:\Users\GZCHE\anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\GZCHE\anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\GZCHE\anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

Wall time: 7min 19s


## Note: A few improvements could be made on this notebook: ##

*1. An improvement to bootstrap algorithm would be i.e. for one vote to sample len(zero)-len(one) = newone, and then concat zero, one, newone, two, newtwo, three, newthree. The benefit of this is that it ensures the original rows are guarenteed to be in the training data at least once.
    Although, it would be rather rate for a row in the original to not be sampled at all*